https://www.kaggle.com/c/nlp-getting-started/data?select=train.csv

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
import re
import nltk
import scipy
import matplotlib.pyplot as plt
import networkx as nx
from gensim.models import word2vec
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline
from nltk.stem.porter import PorterStemmer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('train.csv')
df.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [3]:
df_test = pd.read_csv('test.csv')

In [4]:
df_test.shape

(3263, 4)

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/krishnadevabhaktuni/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [105]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0, len(df)):
    #review = re.sub(r'^https?:\/\/.*[\r\n]*', '', df['text'][i], flags=re.MULTILINE)
    #review = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', df['text'][i], flags=re.MULTILINE)
    review = re.sub(r"http\S+", "", df['text'][i])
    review = re.sub('[^a-zA-Z\d+]', ' ', review)#how can I keep the numbers
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = [lemmatizer.lemmatize(word, pos = 'v') for word in review]
    review = [lemmatizer.lemmatize(word, pos = 'n') for word in review]
    review = ' '.join(review)
    corpus.append(review)

In [106]:
corpus

['deed reason earthquak may allah forgiv u',
 'forest fire near la rong sask canada',
 'resid ask shelter place notifi offic evacu shelter place order expect',
 '13 000 peopl receiv wildfir evacu order california',
 'get send photo rubi alaska smoke wildfir pour school',
 'rockyfir updat california hwi 20 close direct due lake counti fire cafir wildfir',
 'flood disast heavi rain caus flash flood street manit colorado spring area',
 'top hill see fire wood',
 'emerg evacu happen build across street',
 'afraid tornado come area',
 'three peopl die heat wave far',
 'haha south tampa get flood hah wait second live south tampa gonna gonna fvck flood',
 'rain flood florida tampabay tampa 18 19 day lose count',
 'flood bago myanmar arriv bago',
 'damag school bu 80 multi car crash break',
 'man',
 'love fruit',
 'summer love',
 'car fast',
 'goooooooaaaaaal',
 'ridicul',
 'london cool',
 'love ski',
 'wonder day',
 'looooool',
 'way eat shit',
 'nyc last week',
 'love girlfriend',
 'cooool',

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [9]:
y = df['target']

In [10]:
df['text'][7610]

'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ'

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [13]:
detect_model = MultinomialNB().fit(X_train, y_train)

In [14]:
y_pred = detect_model.predict(X_test)

In [15]:
confusion = confusion_matrix(y_test, y_pred)

In [16]:
confusion

array([[761, 125],
       [182, 455]])

In [17]:
accuracy = accuracy_score(y_test, y_pred)

In [18]:
accuracy

0.7984241628365069

In [19]:
lr = LogisticRegression()
print(X_train.shape, y_train.shape)
train = lr.fit(X_train, y_train)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(6090, 2500) (6090,)
Training set score: 0.8985221674876848

Test set score: 0.8063033486539725


In [20]:
# Creating the TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer
cv1 = TfidfVectorizer()
X_td = cv1.fit_transform(corpus).toarray()

In [21]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_td, y, test_size = 0.20, random_state = 0)

In [22]:
tf_model = MultinomialNB().fit(X_train1, y_train1)
y_pred1 = tf_model.predict(X_test1)
confusion_td = confusion_matrix(y_test1, y_pred1)
print(confusion_td)
accuracy1 = accuracy_score(y_test1, y_pred1)
print(accuracy1)

[[800  86]
 [224 413]]
0.7964543663821405


In [23]:
train1 = lr.fit(X_train1, y_train1)
print('Training set score:', lr.score(X_train1, y_train1))
print('\nTest set score:', lr.score(X_test1, y_test1))

Training set score: 0.8939244663382594

Test set score: 0.8049901510177282


In [24]:
text1 = df['text']

In [25]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/krishnadevabhaktuni/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
text = df['text']

In [75]:
nlp = spacy.load('en_core_web_sm')
text_doc = nlp('text')

In [33]:
corpus_spacy = []
for i in range(0, len(df)):
    text_doc = nlp(text[i])
    corpus_spacy.append(text_doc)

In [34]:
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()
tweet_tokens = []
for sent in corpus:
    print(tweet_tokenizer.tokenize(sent))
    tweet_tokens.append(tweet_tokenizer.tokenize(sent))

['deed', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'u']
['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada']
['resident', 'asked', 'shelter', 'place', 'notified', 'officer', 'evacuation', 'shelter', 'place', 'order', 'expected']
['people', 'receive', 'wildfire', 'evacuation', 'order', 'california']
['got', 'sent', 'photo', 'ruby', 'alaska', 'smoke', 'wildfire', 'pours', 'school']
['rockyfire', 'update', 'california', 'hwy', 'closed', 'direction', 'due', 'lake', 'county', 'fire', 'cafire', 'wildfire']
['flood', 'disaster', 'heavy', 'rain', 'cause', 'flash', 'flooding', 'street', 'manitou', 'colorado', 'spring', 'area']
['top', 'hill', 'see', 'fire', 'wood']
['emergency', 'evacuation', 'happening', 'building', 'across', 'street']
['afraid', 'tornado', 'coming', 'area']
['three', 'people', 'died', 'heat', 'wave', 'far']
['haha', 'south', 'tampa', 'getting', 'flooded', 'hah', 'wait', 'second', 'live', 'south', 'tampa', 'gonna', 'gonna', 'fvck', 'flooding']
['raining', 'floo

['hate', 'people', 'tweet', 'receipt', 'know', 'wrong', 'wont', 'take', 'bc', 'blew', 'literally', 'gtfo', 'desperate']
['think', 'blew', 'hopeinhearts', 'notification', 'go', 'check', 'encouraging', 'love']
['hw', 'going', 'dude', 'keeping', 'took', 'exit', 'pulled', 'side', 'told', 'blew', 'motor', 'lolol', 'fast', 'furious']
['universe', 'might', 'actually', 'exist', 'scientist', 'say', 'http', 'co', 'defj', 'xekgx', 'sun', 'blew', 'earth', 'began']
['max', 'blew', 'tf', 'shot', 'fired', 'catfishmtv']
['rick', 'morty', 'blew', 'http', 'co', 'uqkx', 'vbium']
['codythompson', 'ty', 'blew', 'motor', 'went', 'flame', 'got', 'ok']
['realized', 'dude', 'onlyftf', 'way', 'blew', 'tusky', 'game', 'robsimss', 'cantmisskid']
['night', 'blew', 'rq']
['blew', 'snapchat', 'reason']
['queen', 'gambit', 'went', 'well', 'anakin', 'blew', 'droid', 'control', 'ship', 'oh', 'well', 'still', 'fun', 'stop', 'next', 'wbc']
['iphooey', 'time', 'ironically', 'michele', 'bachmann', 'brought', 'w', 'ron', 'p

['new', 'lady', 'shoulder', 'tote', 'handbag', 'faux', 'leather', 'hobo', 'purse', 'cross', 'body', 'bag', 'woman', 'http', 'co', 'zujwuiomb', 'http', 'co', 'gbctmhx', 'pw']
['new', 'summer', 'long', 'thin', 'body', 'bag', 'hip', 'word', 'skirt', 'blue', 'http', 'co', 'jymd', 'ypsj', 'http', 'co', 'pkmmcadg']
['new', 'lady', 'shoulder', 'tote', 'handbag', 'faux', 'leather', 'hobo', 'purse', 'cross', 'body', 'bag', 'woman', 'http', 'co', 'uoozxaus', 'http', 'co', 'pw', 'nbljky', 'rt', 'en']
['louis', 'vuitton', 'monogram', 'sophie', 'limited', 'edition', 'clutch', 'cross', 'body', 'bag', 'full', 'read', 'ebay', 'http', 'co', 'aogcsoy', 'http', 'co', 'djiwg', 'pxv']
['new', 'woman', 'handbag', 'faux', 'leather', 'lady', 'shoulder', 'tote', 'cross', 'body', 'bag', 'large', 'satchel', 'full', 'http', 'co', 'ncjpgf', 'znv', 'http', 'co', 'gerjau', 'ey']
['new', 'lady', 'shoulder', 'tote', 'handbag', 'faux', 'leather', 'hobo', 'purse', 'cross', 'body', 'bag', 'woman', 'http', 'co', 'zujwuiom

['guardian', 'view', 'hiroshima', 'legacy', 'still', 'shadow', 'bomb', 'editorial', 'world', 'longs', 'ca', 'http', 'co', 'ct', 'jutvytg']
['mf', 'life', 'vocal', 'lyrical', 'bomb', 'saw', 'live', 'summer', 'amazing', 'vocalist', 'rt', 'themale', 'madonna', 'melanie', 'fiona', 'slept']
['e', 'hutch', 'da', 'bomb', 'http', 'co', 'aqmpxzo', 'v']
['jay', 'rocking', 'mlb', 'joeybats', 'bombed', 'one', 'rogers', 'centre', 'play', 'offs', 'r', 'ahead', 'bluejays', 'bell', 'moseby', 'barfield', 'r', 'back']
['stankyboy', 'photo', 'bombed']
['sweetiebirks', 'mirrorlady', 'slatukip', 'name', 'country', 'invaded', 'bombed', 'aside', 'libya', 'north', 'africa', 'last', 'year']
['u', 'bombed', 'hiroshima', 'year', 'ago', 'today', 'look', 'war', 'changed', 'since', 'http', 'co', 'uqnj', 'nk', 'http', 'co', 'qlnnmxzfqk']
['london', 'life', 'photo', 'beautiful', 'bombed', 'britain', 'http', 'co', 'racaivffq', 'art']
['believe', 'myfriendmina', 'photo', 'bombed', 'screenshot']
['u', 'drone', 'bomb', '

['cyclist', 'collided', 'runner', 'roanoke', 'greenway', 'win', 'civil', 'verdict', 'roanoke', 'time', 'cyclist', 'c', 'http', 'co', 'e', 'wfgp', 'jhk']
['watch', 'pvris', 'sad', 'bc', 'collided', 'another', 'set']
['motorcyclist', 'bicyclist', 'injured', 'denver', 'collision', 'broadway', 'least', 'two', 'people', 'taken', 'local', 'http', 'co', 'acrgdqhj']
['zojadelin', 'literally', 'almost', 'head', 'collision', 'u', 'today', 'pilot', 'knob']
['north', 'sac', 'elkhorn', 'blvd', 'walerga', 'rd', 'trfc', 'collision', 'enrt', 'http', 'co', 'w', 'ofcc', 'wq']
['beat', 'g', 'motor', 'vehicle', 'collision', 'hit', 'run', 'rainier', 'av', 'charles', 'st', 'reported', 'pm', 'call']
['beat', 'b', 'motor', 'vehicle', 'collision', 'n', 'st', 'fremont', 'av', 'n', 'reported', 'pm', 'call']
['south', 'sac', 'florin', 'rd', 'franklin', 'blvd', 'trfc', 'collision', 'enrt', 'http', 'co', 'e', 'b', 'lyway']
['collision', 'hwy', 'eb', 'east', 'hwy', 'cambridge', 'single', 'vehicle', 'blocking', 'left

...

['refugio', 'oil', 'spill', 'may', 'costlier', 'bigger', 'projected', 'http', 'co', 'rqu', 'ub', 'plf']
['refugio', 'oil', 'spill', 'may', 'costlier', 'bigger', 'projected', 'http', 'co', 'osjurfddkt']
['refugio', 'oil', 'spill', 'may', 'costlier', 'bigger', 'projected', 'http', 'co', 'lzob', 'qoh', 'b']
['refugio', 'oil', 'spill', 'may', 'costlier', 'bigger', 'projected', 'http', 'co', 'bieyguqpb']
['sydney', 'traffic', 'hazard', 'oil', 'spill', 'bankstown', 'stacey', 'st', 'wattle', 'st', 'sydtraffic', 'trafficnetwork']
['plain', 'american', 'pipeline', 'company', 'may', 'spilled', 'crude', 'oil', 'previously', 'estimated', 'ksbynews', 'lilitan', 'http', 'co', 'pegibiqk', 'w']
['syd', 'traffic', 'hazard', 'oil', 'spill', 'bankstown', 'stacey', 'st', 'wattle', 'st', 'http', 'co', 'tzyhdbw', 'f']
['watch', 'video', 'wendell', 'berry', 'speaking', 'bp', 'oil', 'spill', 'gulf', 'mexico', 'birthday', 'http', 'co', 'tn', 'ax', 'xmbb']
['national', 'briefing', 'west', 'california', 'spring'

['seriously', 'tactical', 'riot', 'headquarters', 'disney', 'marvel']
['dani', 'riot', 'keep', 'eye', 'looking', 'lot', 'new', 'team', 'member', 'coming', 'day', 'week']
['meat', 'loving', 'feminist', 'world', 'riot', 'grill', 'arrived', 'pop', 'quiz', 'prefer', 'feminist', 'http', 'co', 'hxox', 'rq']
['live', 'workshop', 'selfie', 'booth', 'riot', 'kappa']
['acaciapenn', 'start', 'big', 'as', 'riot', 'send', 'jail', 'today', 'mf', 'shidddd']
['riot', 'kit', 'bah', 'part', 'new', 'concept', 'gear', 'coming', 'autumn', 'winter', 'menswear', 'fashion', 'urbanfashion', 'http', 'co', 'ccwzdtfbus']
['abran', 'caballero', 'discovered', 'nickcannon', 'listen', 'buy', 'realmandyrain', 'riot', 'itunesmusic', 'itunes', 'http', 'co', 'dehmym', 'lpk', 'blowmandyup']
['liked', 'youtube', 'video', 'http', 'co', 'fr', 'tpzte', 'thorin', 'thought', 'riot', 'sandbox', 'mode', 'lol']
['stuart', 'broad', 'take', 'eight', 'joe', 'root', 'run', 'riot', 'aussie', 'stuart', 'broad', 'took', 'career', 'best',

['investigator', 'rule', 'catastrophic', 'structural', 'failure', 'resulted', 'http', 'co', 'qu', 'iug', 'e', 'r']
['virgin', 'galactic', 'crash', 'early', 'unlocking', 'brake', 'triggered', 'structural', 'failure', 'crash', 'virgin', 'galactic', 'http', 'co', 'x', 'vqxdouvt']
['investigator', 'rule', 'catastrophic', 'structural', 'failure', 'resulted', 'virg', 'related', 'article', 'http', 'co', 'cy', 'lfenyv']
['ntsb', 'virgin', 'galactic', 'crash', 'caused', 'structural', 'failure', 'triggered', 'brake', 'unlocked', 'early', 'http', 'co', 'iknyok', 'zzr', 'http', 'co', 'qckqcx', 'hw']
['investigator', 'say', 'fatal', 'virgin', 'galactic', 'spaceship', 'crash', 'last', 'year', 'caused', 'structural', 'failure', 'co', 'pilot', 'unlocked', 'braking']
['foxnewsinsider', 'obama', 'giving', 'false', 'time', 'schedule', 'iran', 'testing', 'first', 'bomb', 'bomb', 'nuclear', 'suicide', 'vest']
['ever', 'think', 'running', 'choice', 'life', 'rembr', 'kid', 'choice', 'wear', 'suicide', 'bomb'

['listening', 'llegaste', 'tu', 'twister', 'el', 'rey']
['white', 'twister', 'black', 'shift', 'knob', 'x', 'thread', 'size', 'http', 'co', 'sqpshaws', 'w', 'http', 'co', 'udlebh', 'uz']
['twister', 'dance', 'game', 'dance', 'console', 'instruction', 'cable', 'pre', 'loaded', 'song', 'http', 'co', 'bcsxyec', 'ji', 'http', 'co', 'dfxi', 'kvx']
['rodarmer', 'dolla', 'say', 'wicked', 'twister', 'running']
['wish', 'personal', 'hair', 'twister']
['opening', 'scene', 'harry', 'potter', 'order', 'phoenix', 'remind', 'anyone', 'else', 'movie', 'twister', 'okay']
['drunk', 'twister', 'hard']
['mrsbinker', 'emiliorivera', 'davidlabrava', 'mine', 'diesel', 'twister', 'small', 'breed', 'strong', 'beautiful', 'pug']
['thebuffshow', 'thetwisterokc', 'boyfriend', 'want', 'see', 'shaniatwain', 'twisterlovesshania', 'http', 'co', 'h', 'taae']
['curl', 'come', 'pretty', 'look', 'like', 'finished', 'shooting', 'movie', 'twister']
['repped', 'fantastic', 'laura', 'milne', 'thejonesesvoice', 'liguistic', 

['science', 'piece', 'wreckage', 'flight', 'mh', 'confirmed', 'r', 'union', 'island', 'possible', 'http', 'co', 'uqva', 'dfbca']
['malaysia', 'pm', 'confirms', 'wreckage', 'belongs', 'mh', 'http', 'co', 'kacrlpjc', 'l', 'http', 'co', 'yjjbntcazy']
['wreckage', 'conclusively', 'confirmed', 'mh', 'malaysia', 'pm', 'investigator', 'family', 'http', 'co', 'kfzvmxj', 'st']
['wreckage', 'conclusively', 'confirmed', 'mh', 'malaysia', 'pm', 'investigator', 'family', 'http', 'co', 'ptevy', 'mt']
['cia', 'plot', 'rolling', 'eye', 'rt', 'ajabrown', 'chinese', 'relative', 'mh', 'victim', 'say', 'reunion', 'island', 'wreckage', 'planted', 'http', 'co', 'wmnb', 'ita', 'p']
['wreckage', 'conclusively', 'confirmed', 'mh', 'malaysia', 'pm', 'investigator', 'family', 'http', 'co', 'sf', 'rgn', 'wo']
['wreckage', 'conclusively', 'confirmed', 'mh', 'malaysia', 'pm', 'investigator', 'family', 'http', 'co', 'ledmvezcol']
['wreckage', 'conclusively', 'confirmed', 'mh', 'malaysia', 'pm']
['related', 'news', '

In [35]:
from collections import Counter

# Utility function to calculate how frequently words appear in the text.
def word_frequencies(tweet_tokens, include_stop=True):
    
    # Build a list of words.
    # Strip out punctuation and, optionally, stop words.
    words = []
    for token in tweet_tokens:
        for j in token:
            print(j)
            print(type(j))
            if not j.is_punct and (not j.is_stop or include_stop):
                words.append(j.text)
            
    # Build and return a Counter object containing word counts.
    return Counter(words)

In [36]:
text_freq = word_frequencies(tweet_tokens).most_common(10)
text_freq

deed
<class 'str'>


AttributeError: 'str' object has no attribute 'is_punct'

In [ ]:
testlist=[['a','b'],['c','d'],['e','f',],['h','i']]

In [ ]:
for i in testlist:
    for j in i:
        print(j)
    

In [42]:
text[2220]

'Back on the beach after the deluge.  Surf camp in motion.  Our Surf Therapy programme kicked off today for... http://t.co/vjsAqPxngN'

In [60]:
corpi = []
for i in range(0, len(df)):
    text1 = re.sub(r"http\S+", "", text[i])
    corpi.append(text1)


In [61]:
corpi

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 'Forest fire near La Ronge Sask. Canada',
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
 '13,000 people receive #wildfires evacuation orders in California ',
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods...",
 "There's an emergency evacuation happening now in the building across the street",
 "I'm afraid that the tornado is coming to our area...",
 'Three people died from the heat wave so far',
 'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA 

In [83]:
nlp = spacy.load('en')
corpus1 = []
for i in corpus:
    text_doc = 

deed reason earthquak may allah forgiv u
forest fire near la rong sask canada
resid ask shelter place notifi offic evacu shelter place order expect
peopl receiv wildfir evacu order california
got sent photo rubi alaska smoke wildfir pour school
rockyfir updat california hwi close direct due lake counti fire cafir wildfir
flood disast heavi rain caus flash flood street manit colorado spring area
top hill see fire wood
emerg evacu happen build across street
afraid tornado come area
three peopl die heat wave far
haha south tampa get flood hah wait second live south tampa gonna gonna fvck flood
rain flood florida tampabay tampa day lost count
flood bago myanmar arriv bago
damag school bu multi car crash break
man
love fruit
summer love
car fast
goooooooaaaaaal
ridicul
london cool
love ski
wonder day
looooool
way eat shit
nyc last week
love girlfriend
cooool
like pasta
end
bbcmtd wholesal market ablaz
alway tri bring heavi metal rt
africanbaz break news nigeria flag set ablaz aba
cri set ab

new nanotech devic abl target destroy blood clot
skyhawkmm traplord fredosantana lilrees hella crazi fight ambul coupl mosh pit
get run ambul lucki justsay randomthought
news twelv fear kill pakistani air ambul helicopt crash til dna
twelv fear kill pakistani air ambul helicopt crash
tanslash wait ambul
fouseytub ok need ambul hahahah good
ambul sprinter automat frontlin vehicl choic lez compliant ebay
pakistan air ambul helicopt crash kill nine
thenissonian rejectdcartoon nissan ok need medic assist call ambul need
em ny emt petit per hour minimum wage em paramed ambul
twelv fear kill pakistani air ambul helicopt crash
twelv fear kill pakistani air ambul helicopt crash
ambul sprinter automat frontlin vehicl choic lez compliant ebay
ambul sprinter automat frontlin vehicl choic lez compliant ebay
kiwi karyn check park lot said last year ambul st john
know way ambul come lt lt
reuter twelv fear kill pakistani air ambul helicopt crash
twelv fear kill pakistani air ambul helicopt crash
amb

photo sketch base tast armageddon episod startrek to
armageddon
aberdeenfc aberdeenfanpag good luck tomorrow night get coeffici point plz armageddon
paddytomlinson armageddon
rohnertparkdp anoth one histori book thank justic depart way paid incom tax yr
vladimir putin issu major warn late escap armageddon
god kingdom heavenli gov rule peopl earth armageddon
l b entertain lot brucewilli movi dvd die hard monkey armageddon sixth ebay auction
let talk goof guild saunder come right stage
karnythia niec gain abil stand get prepar toddler apocalyps armageddon
check prepper doomsday must librari collect cd shtf preppertalk surviv prepper via ebay
erker eep thought yesterday saw hella scari hail armageddon
ahamedi think messiah alreadi come year ago armageddon dajaal gog amp magog
sadli window reveal microsoft ethic armageddon
armageddon avert el patron ultimalucha
samihonkonen time hour latest seri ww blueprint armageddon extrem impress
european fitba till christma armageddon
christian unit i

infowar nashvil theater attack gun grabber demand hatchet control nwo
anxieti attack
dog attack food pugprob
cop injur gunfight milit attack udhampur polic post suspect milit attack polic post
envw nickcocofre juliedicaro jdabe ask feel attack juli ask frail
messeymetoo feel attack
feel attack
black men make way white men black men get attack
cant believ fuck ci femal go somehow claim offend transgend femal attack medium
isra helicopt attack civilian gaza complet exercis greec
christian attack muslim templ mount wave isra flag via pamela geller
christian attack muslim templ mount wave isra flag via pamela geller
christian attack muslim templ mount wave isra flag via pamela geller
pt unit attack respons target muslim scholar imprison youth
telnet attack streamyx home southern
christian attack muslim templ mount wave isra flag via pamela geller
tbt rememb time patrick kane attack cab driver
im feel attack
ik troll pol rival never liter abus attack famili liter abus ik looser
christian at

fight bioterror sir
cdc pretti cool list bioterror agent
govern expert concern possibl bioterror use gm organ scientist concern
prepar u hh select region special pathogen treatment center bioterror infecti ebola
stationcdrkelli support sy usagov auth taken hostag blk u clergyforc exist younger amp grossli disfigur bioterror ap
amp think possibl full transform imposs quit like medic mysteri bioterror suck
fight bioterror sir
book describ futur therapi technolog sport sexual bioterror diagnosi digitalhealth hcsm
aphiabeta w ugli due ugli amesocialact frat bioterrorismi fbi id u tolew g anoth home abc
howardu blk amp wht collud take wht f usagov auth hostag amp make look blk w bioterror amp use lgl org idi id still
fight bioterror sir
cspanwj blk amp wht collud take wht f usagov auth hostag amp make look blk w bioterror amp use lgl org idi id still
onu franc bioterror rockefel chi rockefelleruniv heiress evad lgl effort prosecut blk harvardu kidnap g afp
irand cover nuclear activ bioterro

...

want twister ticket chanc vip experi see shania click
stop fuck say whole nother sound fuck stupid fuck mean whole fuck tongu twister
landslid tumbl todayi side twister sweep away bitch run distanc
love twister thetwist maddi tae oktxduo
brain twister homefolk opinion propos modern canada omw
crazi mom threw teen daughter nude twister sex parti accord friend pic
sharper imag viper hardsid twister black
twister fun
brain twister let drop telli structur cast edcxo
bed eat twister drink cup tea dunk twister tea would well weird
sarahmcp justjon give titti twister
last second outbid rt catlow c twister swivel dc fuel line made usa plea favorit amp shar
reason brain twister oneself discount redesign websit itrawcwb
check want twister ticket vip experi see shania click would love love love win
jam riser dierk bentley thetwist iheartradio nowplay
bulgarian titti twister noemot produc edk pathfind hd via youtub
ellenfromnowon speed nexu shifter commun cargo bike
anyon wanna come watch twister 

japan aogashima volcano unknown check aogashima japan photographi volcano
earthquak sismo km volcano hawaii time utc
maailiss diaporama sixpencee karymski lake crater lake locat karymski volcano russia
earthquak occur km volcano hawaii utc earthquak volcano
earthquak km volcano hawaii iphon user download earthquak app inform
erupt indonesian volcano spark transport chao pictur done video mount raung
shoe asic gt ii super red ronni fieg kith red white x gel grey volcano
usg report earthquak km volcano hawaii utc quak
usg eq km volcano hawaii time utc earthquak
usg km volcano hawaii time utc epicent sm
mrmikeeaton muazimu prime hill hill mountain volcano hell mountain hill hil
diaporama sixpencee karymski lake crater lake locat karymski volcano russia
utc km volcano hawaii
shoe asic gt ii super red ronni fieg kith red white x gel grey volcano
songhey well also gay girl like predict tsunami amp volcano amp earthquak gift god christian
ad video youtub playlist doc volcano earthquak insid v

ckosova read whirlwind subject amongst other bomb save million live ye true
edfring highli recommend nf l titania summerhalleri whirlwind reimagin shakespear midsumm
whirlwind scourg europ rt whedonesqu could watch fang four
whirlwind weekend start hour cantwaittoplayinminneapoli
week whirlwind exactli imagin nyc version career like
girl got girlfriend chevi blue like whirlwind
picthi thank much toronto whirlwind amazing glad final
past week absolut whirlwind athen bound
win lisowski whitewash whirlwind round shanghai master qual snooker
wild fire blacken northern california parch harri counti becom tinder box
collect artic video west coast wild fire abc
brother n law riooooo got call head north fight wild fire dude beast
sever wild fire burn lot land california one larger fire
map show california wild fire burn map creat cal fire
great footag strong work san bernardino counti fire also work wild fire right
god forbid true california enough problem sever drought amp wild fire nuclear p

latestnew polic offic wound suspect dead exchang shot richmond va ap richmond pol
abc news polic offic wound suspect dead exchang shot news
nation free root beer float day tomorrow amp w pm close help support wound warrior project donat
small bag bottom wound hero shake senzu bean fall gohan surpris shake pouch
wocowa offic wound suspect kill exchang gunfir ush
polic offic wound suspect dead exchang shot richmond polic offic wound suspect kill
gunshot wound bicep one ten wound chest torso area kerricktri jonathanferrel
charlesdagnal get think salt wound rub
nout like rubbin salt wound dad
read unheal wound unend conflict kashmir pakistan india endoccup endconflict freekashmir
would becom mirror reflect privat wound desir
acc studi conduct skim morethan popul kashmir suffer psychiatr disord kashmirconflict
equal scare caus somehow start heal fill wound thought perman
rt dianneg gunshot wound bicep wound chest torso area kerricktri jonathanferrel
white ppl bruis easili ur bullet wound
he

rt livingsaf nw issu sever thunderstorm warn part ar nc ok expect trauma case
mh aircraft debri found la reunion miss malaysia airlin
father three lost control car overtak collid bathandnortheastsomerset
earthquak km ssw anza california iphon user download earthquak app inform
evacu order lift town roosevelt
break la refugio oil spill may costlier bigger project
siren went forney tornado warn
offici say quarantin place alabama home possibl ebola case develop symptom
worldnew fallen powerlin g link tram updat fire crew evacu passeng tr
flip side walmart bomb everyon evacu stay tune blow
suicid bomber kill saudi secur site mosqu reuter via world googl news wall
stormchas violent record break ef el reno oklahoma tornado nearli run
two giant crane hold bridg collaps nearbi home
aria ahrari thetawniest control wild fire california even northern part state troubl
utc km volcano hawaii
polic investig e bike collid car littl portug e bike rider suffer seriou non life threaten injuri
latest hom

In [84]:
text

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [86]:
lemmatizer.lemmatize("earthquake")

'earthquake'

In [99]:
wordnet_lemmatizer = WordNetLemmatizer()
wordnet_lemmatizer.lemmatize("earthquak", pos="n")

'earthquak'